<a href="https://colab.research.google.com/github/SvisBee/3.34/blob/master/GenTestBD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Faker

In [ ]:
import random
import csv
from datetime import datetime, timedelta
from faker import Faker

In [ ]:
# Русской локализацией для Faker
fake = Faker("ru_RU")

roles_map = {
    "visitor": 1,
    "candidate": 2,
    "external_user": 3
}

statuses_map = {
  "предзаявка": 1,
  "на рассмотрении": 2,
  "отклонена": 3,
  "принята": 4
}

semesters_data = [
  {"semester_id": 1, "start_date": datetime(2024, 2, 1), "end_date": datetime(2024, 5, 31)},
  {"semester_id": 2, "start_date": datetime(2024, 9, 1), "end_date": datetime(2024, 12, 31)}
]

courses_map = {
  "Java development": 1,
  "Fronted development": 2,
  "Testing": 3,
  "Analitics": 4,
  "DevOps Engineer": 5,
  "Data Engineer": 6,
  "System Engineer": 7,
  "Data Science": 8,
  "Test-Analyst": 9
}

In [ ]:
def random_datetime_in_range(start, end):
    """Выбирает случайное время между start и end."""
    delta = end - start
    random_days = random.randrange(delta.days + 1)
    random_seconds = random.randrange(24 * 60 * 60)
    return start + timedelta(days=random_days, seconds=random_seconds)

def generate_users(num_users):
    """Генерация списка пользователей."""
    users = []
    role_names = list(roles_map.keys())  # ['visitor', 'candidate', 'external_user']
    role_weights = [0.1, 0.2, 0.7]  # Соответствующие вероятности

    for i in range(1, num_users+1):
        #user_id = f"U{i:04d}"  # 'U0001', 'U0002', ...
        user_id = fake.numerify("##########")  # 10 случайных цифр, например "9274031850"

        #first_name = fake.first_name()
        #last_name = fake.last_name()

        # Определяем, мужчина или женщина
        is_male = random.random() < 0.5
        if is_male:
            first_name = fake.first_name_male()
            last_name = fake.last_name_male()
        else:
            first_name = fake.first_name_female()
            last_name = fake.last_name_female()

        phone = fake.numerify("9#########")  # 10 цифр
        email = fake.email()
        city = fake.city_name()

        # Выбираем роль с заданными вероятностями
        role_name = random.choices(role_names, weights=role_weights, k=1)[0]
        role_id = roles_map[role_name]

        users.append({
            "user_id": user_id,
            "first_name": first_name if role_id != 1 else "",
            "last_name": last_name if role_id != 1 else "",
            "phone": phone if role_id != 1 else "",
            "email": email if role_id != 1 else "",
            "role_id": role_id,
            "city": city if role_id != 1 else ""
        })
    return users

def generate_applications(users):
    """Генерация заявок (applications) для пользователей."""

    applications = []
    app_id_counter = 1

    for user in users:
        if user["role_id"] == 1:
            # Пользователи с ролью 1 (visitor) не имеют заявок
            continue

        else:
            # Определяем возможные статусы в зависимости от роли
            if user["role_id"] == 3: # external_user
                # Для role_id 3 исключаем статус 'предзаявка' (1)
                possible_statuses = ["на рассмотрении", "отклонена", "принята"]
            else:
                possible_statuses = ["предзаявка"]

            # Выбираем начальный статус из возможных
            initial_status = random.choice(possible_statuses)
            status_id = statuses_map[initial_status]

            # Берём случайный курс из словаря
            # Если это 'предзаявка', то у заявки не может быть course_id
            if status_id == 1:
                c_id = None
            else:
                # Выбираем случайный курс
                _, c_id = random.choice(list(courses_map.items()))

            # Выбираем семестр и генерируем даты
            semester = random.choice(semesters_data)
            creation_dt = random_datetime_in_range(semester["start_date"], semester["end_date"])

            # change_time = creation_time + случайный интервал
            change_dt = creation_dt + timedelta(days=random.randint(0, 10))

            applications.append({
                "app_id": app_id_counter,
                "user_id": user["user_id"],
                "course_id": c_id, # может быть None
                "status_id": status_id,
                "creation_time": creation_dt.strftime("%Y-%m-%d %H:%M:%S"),
                "change_time": change_dt.strftime("%Y-%m-%d %H:%M:%S"),
                "semester_id": semester["semester_id"]
            })
            app_id_counter += 1
    return applications

def generate_status_changes(applications, prob_change=0.4):
    """
    Генерирует историю смены статусов (status_changes).
    С вероятностью prob_change заявка переходит на новый статус ("принята"/"отклонена").
    """
    changes = []
    change_id_counter = 1

    for app in applications:
        current_status = app["status_id"]

        if current_status != 2:
          continue
        else:
            old_status_id = current_status

            # С вероятностью prob_change изменяем статус на "принята" или "отклонена"
            if random.random() < prob_change:
                new_status_name = random.choice(["принята", "отклонена"])
                new_status_id = statuses_map[new_status_name]

                changes.append({
                    "change_id": change_id_counter,
                    "app_id": app["app_id"],
                    "old_status_id": old_status_id,
                    "new_status_id": new_status_id
                })
                change_id_counter += 1

                # Обновляем заявку
                app["status_id"] = new_status_id

    return changes

def save_csv(data_list, filename, fieldnames):
    """
    Сохранения списка словарей в CSV.
    """
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data_list)

In [ ]:
# Генерируем пользователей

'''Указание количества пользователей'''
users = generate_users(num_users=5000)
'''---------------------------------'''

save_csv(
    users,
    filename="users_test_data.csv",
    fieldnames=["user_id", "first_name", "last_name", "phone", "email", "role_id", "city"]
)
print(f"Сохранено {len(users)} пользователей в users_test_data.csv")

# Генерируем заявки
applications = generate_applications(users)
save_csv(
    applications,
    filename="applications_test_data.csv",
    fieldnames=["app_id","user_id","course_id","status_id","creation_time","change_time","semester_id"]
)
print(f"Сохранено {len(applications)} заявок в applications_test_data.csv")

# Генерируем историю смены статусов
status_changes_list = generate_status_changes(applications, prob_change=0.4)
save_csv(
    status_changes_list,
    filename="status_changes_test_data.csv",
    fieldnames=["change_id","app_id","old_status_id","new_status_id"]
)
print(f"Сохранено {len(status_changes_list)} смен статусов в status_changes_test_data.csv")

# Перезапишем applications с учётом новых статусов
save_csv(
    applications,
    filename="applications_test_data_final.csv",
    fieldnames=["app_id","user_id","course_id","status_id","creation_time","change_time","semester_id"]
)
print("Обновлённые заявки сохранены в applications_test_data_final.csv")

Сохранено 5000 пользователей в users_test_data.csv
Сохранено 4521 заявок в applications_test_data.csv
Сохранено 445 смен статусов в status_changes_test_data.csv
Обновлённые заявки сохранены в applications_test_data_final.csv
